In [1]:
# Parameters
cell_type = "B"


In [2]:
import sys
import os

import anndata as ad

import pandas as pd
import numpy as np

from pyprojroot import here

import re

from tqdm import tqdm

In [3]:
def load_sorted_shap_values_fnames(
    cell_type: str = '',
    run_name: str = 'run1'):

    dirpath = here(f"inflammabucket_bkp/03_downstream_analysis/05_SHAP/results/04_shap/shap_vals")
    fname_regex = f'^{run_name}_{cell_type}_shap_values_' + r'(\d+)'
    results_batches = [
        (fname, int(re.search(fname_regex, fname).group(1)))
        for fname in os.listdir(dirpath)
        if re.search(fname_regex, fname)]

    sorted_files = sorted(results_batches, key=lambda x: x[1])

    sorted_filenames = [os.path.join(dirpath, filename) for filename, _ in sorted_files]

    return sorted_filenames

In [4]:
shap_val_list = [np.load(fname)['shap_values'] for fname in tqdm(load_sorted_shap_values_fnames(cell_type))] 


  0%|          | 0/36 [00:00<?, ?it/s]

  3%|▎         | 1/36 [00:03<02:03,  3.52s/it]

  6%|▌         | 2/36 [00:07<01:59,  3.51s/it]

  8%|▊         | 3/36 [00:10<01:55,  3.51s/it]

 11%|█         | 4/36 [00:14<01:53,  3.54s/it]

...

 92%|█████████▏| 33/36 [01:57<00:10,  3.56s/it]

 94%|█████████▍| 34/36 [02:01<00:07,  3.55s/it]

 97%|█████████▋| 35/36 [02:04<00:03,  3.54s/it]

100%|██████████| 36/36 [02:07<00:00,  3.23s/it]

100%|██████████| 36/36 [02:07<00:00,  3.53s/it]

In [5]:
adata = ad.read_h5ad(here(f'inflammabucket_bkp/03_downstream_analysis/05_SHAP/data/{cell_type}_adataMerged_SPECTRAgenes.log1p.h5ad'), backed='r')

In [6]:
## Gene symbols and Disease labels
symbols_df = pd.read_pickle(here('inflammabucket_bkp/03_downstream_analysis/04_selected_gene_list.pkl'))
symbols_sorted = symbols_df.loc[adata.var_names].symbol.values

DISEASES = ['BRCA', 'CD', 'COPD', 'COVID', 'CRC', 'HBV', 'HIV', 'HNSCC', 'MS', 'NPC', 'PS', 'PSA', 'RA', 'SLE', 'UC', 'asthma', 'cirrhosis', 'flu', 'healthy', 'sepsis']

diseaseDict = dict()
for d in DISEASES:
    diseaseDict[d] = []

In [7]:
shape_values_matrix = np.concatenate(shap_val_list)

In [8]:
for idx, values in tqdm(adata.obs.groupby('sampleID', observed=True).indices.items()):
    geneXdisease_sample_i = pd.DataFrame(shape_values_matrix[values].mean(0))
    geneXdisease_sample_i.columns = DISEASES
    geneXdisease_sample_i.index = symbols_sorted
    for d in geneXdisease_sample_i.columns:
        diseaseDict[d].append(pd.DataFrame.from_dict({idx:geneXdisease_sample_i[d]}))

  0%|          | 0/814 [00:00<?, ?it/s]

  2%|▏         | 16/814 [00:00<00:05, 153.12it/s]

  4%|▍         | 35/814 [00:00<00:04, 172.82it/s]

  7%|▋         | 53/814 [00:00<00:04, 168.40it/s]

  9%|▊         | 71/814 [00:00<00:04, 167.55it/s]

...

 92%|█████████▏| 745/814 [00:05<00:00, 176.10it/s]

 94%|█████████▍| 764/814 [00:05<00:00, 176.26it/s]

 96%|█████████▋| 785/814 [00:05<00:00, 184.22it/s]

 99%|█████████▉| 804/814 [00:06<00:00, 147.63it/s]

100%|██████████| 814/814 [00:06<00:00, 131.24it/s]

In [9]:
for d in tqdm(DISEASES):
    pd.concat(diseaseDict[d], axis=1).to_csv(here(f'03_downstream_analysis/08_gene_importance/new_shap_plots/results/SHAP_AVGsamples/SHAP_AVGsample_{cell_type}_{d}.csv'))

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:00<00:10,  1.80it/s]

 10%|█         | 2/20 [00:01<00:10,  1.72it/s]

 15%|█▌        | 3/20 [00:01<00:10,  1.61it/s]

 20%|██        | 4/20 [00:02<00:11,  1.39it/s]

...

 85%|████████▌ | 17/20 [00:10<00:01,  1.54it/s]

 90%|█████████ | 18/20 [00:11<00:01,  1.40it/s]

 95%|█████████▌| 19/20 [00:12<00:00,  1.35it/s]

100%|██████████| 20/20 [00:13<00:00,  1.13it/s]

100%|██████████| 20/20 [00:13<00:00,  1.47it/s]